# Exploring in-process query options for Parquet 
This notebook explores using datafusion and DuckDB for querying Parquet data files holding NWM, xref, and usgs actual data.

## Getting initial parquet data for the NWM

This first code block will run for a while and generate the parquet files for the time period specified in the script.

The code block then moves the parquet files to data/nwm/ to keep track of which parquet files are associated with particular output; as there will be multiple sets of parquet files.


In [ ]:
# this was actually run from a bash prompt prior to the current placement into the explore folder
# may need to be tweaked to get the path right
!cd ../hydro-evaluation/parquet
!#python3 nwm_to_parquet.py #commented out so it isn't run by accident
! cd ../..
! mkdir -p data/nwm/
!#mv parquet/*.parquet data/nwm/ #prevent run by accident


In [ ]:
# adding project dirs to path so code may be referenced from the notebook
import sys
sys.path.insert(0, '../hydro-evaluation/wide_table')
sys.path.insert(0, '../hydro-evaluation')

In [ ]:
import utils
import importlib
importlib.reload(utils)

In [ ]:
file = "../hydro-evaluation/data/RouteLink_CONUS_NWMv2.1.6.csv"
xwalk_data = utils.get_xwalk(file)
xwalk_data.head()

In [ ]:
from pathlib import Path
import os
xwalk_data2 = utils.get_xwalk() #tests original method without file path
xwalk_data2.head()

In [ ]:
!pip3 install pyarrow

In [ ]:
xwalk_data.to_parquet("../hydro-evaluation/data/xwalk.parquet")

In [ ]:
%env HDF5_DIR=/opt/homebrew/opt/hdf5
!pip3 install hydrotools

In [ ]:
# replicating the ingest_usgs code here so the data may be
# saved as parquet data rather than inserted into the timeseriesDB
from hydrotools.nwis_client.iv import IVDataService
from datetime import datetime, timedelta
from insert_usgs import fetch_usgs

start = datetime(2022, 10, 1)
download_period = timedelta(days=1)
number_of_periods = 30

for p in range(number_of_periods):
    print("Processing: ", p)
    start_dt = (start + download_period * p)
    end_dt = (start + download_period * (p + 1))
    start_dt_str = start_dt.strftime("%Y-%m-%d")
    end_dt_str = end_dt.strftime("%Y-%m-%d")

    observations_data = fetch_usgs(
        start_dt=start_dt_str,
        end_dt=end_dt_str
    )
    
    observations_data.set_index("value_time", inplace=True)
    obs = observations_data[
        observations_data.index.hour.isin(range(0, 23)) 
        & (observations_data.index.minute == 0) 
        & (observations_data.index.second == 0)
    ]
    obs.reset_index(level=0, allow_duplicates=True, inplace=True)
    obs.to_parquet("../hydro-evaluation/data/usgs/" + str(p) + ".parquet")


# loading sampe data from TimescaleDB

In [ ]:
import pandas as pd
query1_head = pd.read_csv("query1_head.csv")

query1_head

In [ ]:
query2_head = pd.read_csv("query2_head.csv")

query2_head

In [ ]:
query3_head = pd.read_csv("query3_head.csv")

query3_head

# Exploring datafusion
A SQL engine on top of various file formats including Parquet; does not yet have enough stats functions to be immediately useful via SQL; 
## results
- consumed all available memory for query1 and then failed; ran for a very long time
- tried to modify query many different ways with same result
- for some reason, query planner did not like simple in statement; couldn't find any documentation for why this would be a problem
- do not recommend as it is too brittle and not as memory efficient as advertised for our use case

In [ ]:
!pip3 install datafusion

In [ ]:
import datafusion as df
ctx = df.SessionContext()

ctx.register_parquet('nd', '../hydro-evaluation/data/nwm/*.parquet')
ctx.register_parquet('nux', '../hydro-evaluation/data/xwalk.parquet')
ctx.register_parquet('ud', '../hydro-evaluation/data/usgs/*.parquet')


In [ ]:
from datafusion import col

query = """
    SELECT nd.reference_time,
        nd.value_time,
        nd.nwm_feature_id,   
        nd.value as forecast_value, 
        nd.configuration,  
        nd.measurement_unit,     
        nd.variable_name,
        nux.latitude,
        nux.longitude,
        ud.value as observed_value,
        ud.usgs_site_code
    FROM nd 
    JOIN nux 
        on nux.nwm_feature_id = nd.nwm_feature_id 
    JOIN ud 
        on nux.usgs_site_code  = ud.usgs_site_code 
        and nd.value_time = ud.value_time 
        and nd.measurement_unit = ud.measurement_unit
        and nd.variable_name = ud.variable_name
                    where nd.nwm_feature_id in (select 6731199,2441678,14586327,8573705,2567762,41002752,8268521,41026212,4709060,20957306)


"""
query = """
    select nd.reference_time,
        nd.value_time,
        nd.nwm_feature_id,   
        nd.value as forecast_value, 
        nd.configuration,  
        nd.measurement_unit,     
        nd.variable_name,
        nux.latitude,
        nux.longitude
    from nd
    join nux 
        on nux.nwm_feature_id = nd.nwm_feature_id
    JOIN ud 
        on nux.usgs_site_code  = ud.usgs_site_code 
        and nd.value_time = ud.value_time 
        and nd.measurement_unit = ud.measurement_unit
        and nd.variable_name = ud.variable_name
    where nd.nwm_feature_id in (select 6731199,2441678,14586327,8573705,2567762,41002752,8268521,41026212,4709060,20957306)
"""
df = ctx.sql(query)
df.show()
#df.filter(col("nwm_feature_id") == 6731199)

# exploring DuckDB

## results
- very efficent use of memory; never much more than the memory allocated for the notebook service
- very fast query execution and fetch times
- query planner had no problems with any of the existing or modified SQL statements
- additional performance tuning possibilities down the road, things similar to materialized views
- integrates really well with Python and other languages

In [ ]:
!pip3 install duckdb

In [ ]:
import duckdb as ddb
con = ddb.connect(database='explore.duckdb')

In [ ]:
query = """
    with joined as (
        SELECT 
        nd.reference_time,
        nd.value_time,
        nd.nwm_feature_id,   
        nd.value as forecast_value, 
        nd.configuration,  
        nd.measurement_unit,     
        nd.variable_name,
        --nux.geom as geom, 
        ud.value as observed_value,
        ud.usgs_site_code,
        nd.value_time - nd.reference_time as lead_time
        FROM '../hydro-evaluation/data/nwm/*.parquet'  nd 
        JOIN '../hydro-evaluation/data/xwalk.parquet'  nux 
        on nux.nwm_feature_id = nd.nwm_feature_id 
        JOIN '../hydro-evaluation/data/usgs/*.parquet'  ud 
        on nux.usgs_site_code  = ud.usgs_site_code 
        and nd.value_time = ud.value_time 
        and nd.measurement_unit = ud.measurement_unit
        and nd.variable_name = ud.variable_name
        where nd.nwm_feature_id in (6731199,2441678,14586327,8573705,2567762,41002752,8268521,41026212,4709060,20957306)
    )
    select 
    reference_time, 
    nwm_feature_id,
    regr_intercept(forecast_value, observed_value) as intercept,
    covar_pop(forecast_value, observed_value) as covariance,
    corr(forecast_value, observed_value) as corr,
    regr_r2(forecast_value, observed_value) as r_squared,
    count(forecast_value) as forecast_count,
    count(observed_value) as observed_count,
    avg(forecast_value) as forecast_average,
    avg(observed_value) as observed_average,
    var_pop(forecast_value) as forecast_variance,
    var_pop(observed_value) as observed_variance,
    sum(observed_value - forecast_value)/count(*) as bias,
    max(forecast_value) - max(observed_value) as max_forecast_delta
    from joined 
    group by reference_time, nwm_feature_id
    order by nwm_feature_id, max_forecast_delta;
"""

query1_df = con.execute(query).df()
query1_df.head()

In [ ]:
bias = query1_df.pivot(index="reference_time", columns="nwm_feature_id", values="bias")
bias.plot(figsize=(20,10))

In [ ]:
nwm_feature_id = 17003262
configuration = 'medium_range_mem1'

query = f"""
    SELECT 
    nd.reference_time,
    nd.nwm_feature_id,   
    nd.value_time,
    regr_intercept(nd.value, ud.value) as intercept,
    covar_pop(nd.value, ud.value) as covariance,
    corr(nd.value, ud.value) as corr,
    regr_r2(nd.value, ud.value) as r_squared,
    count(nd.value) as forecast_count,
    count(ud.value) as observed_count,
    avg(nd.value) as forecast_average,
    avg(ud.value) as observed_average,
    var_pop(nd.value) as forecast_variance,
    var_pop(ud.value) as observed_variance,
    max(nd.value) - max(ud.value) as max_forecast_delta,
    sum(ud.value - nd.value)/count(*) as bias
    FROM '../hydro-evaluation/data/nwm/*.parquet'  nd 
    JOIN '../hydro-evaluation/data/xwalk.parquet'  nux 
        on nux.nwm_feature_id = nd.nwm_feature_id 
    JOIN '../hydro-evaluation/data/usgs/*.parquet'  ud 
        on nux.usgs_site_code  = ud.usgs_site_code 
        and nd.value_time = ud.value_time 
        and nd.measurement_unit = ud.measurement_unit
        and nd.variable_name = ud.variable_name
    where nd.nwm_feature_id = {nwm_feature_id}
    and configuration = '{configuration}'
    group by nd.reference_time,
    nd.nwm_feature_id,   
    nd.value_time
"""
query2_df = con.execute(query).df()
query2_df.head()

In [ ]:
query = """
SELECT 
    nd.reference_time,
    nd.value_time,
    nd.nwm_feature_id,   
    nd.value as forecast_value, 
    nd.configuration,  
    nd.measurement_unit,     
    nd.variable_name,
    nux.latitude,
    nux.longitude,
    ud.value as observed_value,
    ud.usgs_site_code,
    nd.value_time - nd.reference_time as lead_time
FROM '../hydro-evaluation/data/nwm/*.parquet'  nd 
JOIN '../hydro-evaluation/data/xwalk.parquet'  nux 
    on nux.nwm_feature_id = nd.nwm_feature_id 
JOIN '../hydro-evaluation/data/usgs/*.parquet'  ud 
    on nux.usgs_site_code  = ud.usgs_site_code 
    and nd.value_time = ud.value_time 
    and nd.measurement_unit = ud.measurement_unit
    and nd.variable_name = ud.variable_name
where configuration = 'medium_range_mem1'
and nd.nwm_feature_id = 17003262
order by reference_time, nd.nwm_feature_id;
"""
query3_df = con.execute(query).df()
query3_df.head()

In [ ]:
import matplotlib.pyplot as plt

ax = plt.gca()
query3_df.plot(x= 'value_time', y="forecast_value", ax = ax, figsize=(20,10))
query3_df.plot(x= 'value_time', y="observed_value", ax = ax, figsize=(20,10))


# Next steps?
- Initial example dashboard with larger data set for DuckDB?
- Dash appears to be a better choice:
    - subjectively dashboard examples look better out of the box on demo sites
    - stack would be easier to customize if needed: Python, Flask, Plotly, React
    - supports multiple languages: Python, R, Julia
- How best to handle geospatial data types?
    - Geospatial data types are not yet part of DuckDB, looks like it is being discussed in their issues/tickets
    - Explore GeoParquet with GeoPandas(extension of Pandas for geospatial types and operations)
    - SpatiaLite as option to investigate handling of geospatial data with another query to DuckDB parquet data
    - GDAL/Parquet handling
    - Potential for a linked server between SpatiaLite and DuckDB with a view in DuckDB to all the Parquet data exposed to SpatiaLite for handling geospatial conversions
    
    